# Oogeso - test case 2
Simple case for illustrating/testing the Oogeso model
* Gas surce
* Gas turbine generators
* Wind turbine, with power gradually increasing and then decreasing
* Electrical loads
* Heat loads

In [38]:
%reload_ext autoreload
%autoreload 2
import oogeso
from oogeso import plots
import IPython
import pyomo.environ as pyo
import pandas as pd
import plotly.express as px
import json

In [39]:
profiles_dfs = oogeso.io.read_profiles_from_csv(
    filename_forecasts="../tests/test_data/testcase2_profiles_forecasts.csv",
    filename_nowcasts="../tests/test_data/testcase2_profiles_nowcasts.csv",timestamp_col="timestamp",exclude_cols=["timestep"])
profiles_json = oogeso.utils.create_time_series_data(profiles_dfs["forecast"],profiles_dfs["nowcast"],
                                      time_start=None,time_end=None,timestep_minutes=15)
data0 = oogeso.io.read_data_from_yaml('../tests/test_data/testcase2_inputdata.yaml')
data0.profiles = [x for x in profiles_json if x.id in ["wind","demand"]]

C:\Users\sondr\Documents\oogeso\src\oogeso\utils\util.py:27: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



In [58]:
simulator = oogeso.Simulator(data=data0)
optimiser = simulator.optimiser

In [59]:
res = simulator.run_simulation(solver="gurobi_persistent",time_range=[0,90],write_yaml=False)

Set parameter TimeLimit to value 10
Set parameter QCPDual to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7600K CPU @ 3.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1084 rows, 3297 columns and 1997 nonzeros
Model fingerprint: 0x37efafbf
Variable types: 2937 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [1e-02, 3e-02]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 5e+02]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint x3946 by 5.000000000

Presolve removed 954 rows and 3203 columns
Presolve time: 0.01s
Presolved: 130 rows, 94 columns, 397 nonzeros
Variable types: 24 continuous, 70 integer (70 binary)
Found heuristic solution: objective 0.1789250
Found heuristic solution: objective 0.1047012
Foun

C:\Users\sondr\Documents\oogeso\src\oogeso\dto\base.py:151: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.

C:\Users\sondr\Documents\oogeso\src\oogeso\dto\base.py:151: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.

C:\Users\sondr\Documents\oogeso\src\oogeso\dto\base.py:151: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.

C:\Users\sondr\Documen

Set parameter TimeLimit to value 10
Set parameter QCPDual to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7600K CPU @ 3.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1084 rows, 3297 columns and 1997 nonzeros
Model fingerprint: 0x7fa61f8d
Variable types: 2937 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [1e-02, 3e-02]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 5e+02]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint x4182 by 3.000000000

Presolve removed 985 rows and 3225 columns
Presolve time: 0.00s
Presolved: 99 rows, 72 columns, 297 nonzeros
Variable types: 20 continuous, 52 integer (52 binary)
Found heuristic solution: objective 0.1623842
Found heuristic solution: objective 0.0880893

Root

In [42]:
save_results = False
if save_results:
    import oogeso.dto.serialisation
    with open("testcase2_resultobject.json", "w") as outfile:
        json.dump(res, fp=outfile, cls=oogeso.dto.serialisation.DataclassJSONEncoder, indent=2)

In [43]:
# Open saved result for comparison
with open("../tests/test_data/testcase2_resultobject.json", "r") as infile:
    res2 = json.load(infile,cls=oogeso.dto.serialisation.OogesoResultJSONDecoder)
    #res2=oogeso.dto.serialisation.deserialize_oogeso_results(json_data=json_data)

## Inspect optimisation problem
We can inspect the optimisation problem to understand what is happening

In [44]:
gt='GT2'
print("{} on/off initial status = {}".format(gt,pyo.value(optimiser.paramDeviceIsOnInitially[gt])))
print("{} timesteps in startup preparation = {}".format(gt,pyo.value(optimiser.paramDevicePrepTimestepsInitially[gt])))
print("start-stop constraint:")
for t in range(4):
    cstr=getattr(optimiser, "constr_{}_startstop".format(gt))[t]
    #cstr=optimiser.constrDevice_startup_shutdown[gt,t]
    print("  t={}: {} = {}".format(t,cstr.body,cstr.upper))
print("start delay constraint:")
for t in range(4):
    cstr=getattr(optimiser, "constr_{}_startdelay".format(gt))[t]
    #cstr=optimiser.constrDevice_startup_delay[gt,t]
    print("  t={}: {} = {}".format(t,cstr.body,cstr.upper))
print("max flow constraint:")
for t in range(4):
    cstr=getattr(optimiser, "constr_{}_flowMax".format(gt))[t]
    #cstr=optimiser.constrDevicePmax[gt,t]
    print("  t={}: {} = {}".format(t,cstr.body,cstr.upper))

GT2 on/off initial status = 0
GT2 timesteps in startup preparation = 0
start-stop constraint:
  t=0: varDeviceIsOn[GT2,0] - paramDeviceIsOnInitially[GT2] + varDeviceStopping[GT2,0] = 0.0
  t=1: varDeviceIsOn[GT2,1] - varDeviceIsOn[GT2,0] + varDeviceStopping[GT2,1] = 0.0
  t=2: varDeviceIsOn[GT2,2] - varDeviceIsOn[GT2,1] + varDeviceStopping[GT2,2] = 0.0
  t=3: varDeviceIsOn[GT2,3] - varDeviceIsOn[GT2,2] - (varDeviceStarting[GT2,0] - varDeviceStopping[GT2,3]) = 0.0
start delay constraint:
  t=0: varDeviceIsPrep[GT2,0] - varDeviceStarting[GT2,0] = 0.0
  t=1: varDeviceIsPrep[GT2,1] - (varDeviceStarting[GT2,1] + varDeviceStarting[GT2,0]) = 0.0
  t=2: varDeviceIsPrep[GT2,2] - (varDeviceStarting[GT2,2] + varDeviceStarting[GT2,1] + varDeviceStarting[GT2,0]) = 0.0
  t=3: varDeviceIsPrep[GT2,3] - (varDeviceStarting[GT2,3] + varDeviceStarting[GT2,2] + varDeviceStarting[GT2,1]) = 0.0
max flow constraint:
  t=0: varDeviceFlow[GT2,el,out,0] - 20.0*varDeviceIsOn[GT2,0] = 0.0
  t=1: varDeviceFlow[GT2,

## Plots

Using `timestep=None` plots the network diagram without results - useful for viewing network before simulation, or for debugging if simulation fails to give results.

In [45]:
dotG = plots.plot_network(simulator,timestep=None)
# Uncomment to show plot (requires graphviz?)
try:
    x=IPython.display.Image(dotG.create_png())
except:
    x=None
x

In [46]:
# Network plot with results at a given timestep
dotG=plots.plot_network(simulator=simulator,timestep=15,filename=None)
try:
    x=IPython.display.Image(dotG.create_png())
except:
    x=None
x

In [47]:
plots.plot_sum_power_mix(res,optimisation_model=simulator.optimiser,carrier="el").show()

In [48]:
plots.plot_device_profile(res,optimisation_model=simulator.optimiser,devs=['GT1','GT2','GT3'],
    include_on_off=True,include_prep=True).update_layout(title="Gas turbine operation")

In [49]:
plots.plot_CO2_rate_per_device(res,optimisation_model=simulator.optimiser,device_shareload=None)

In [50]:
plots.plot_reserve(res,simulator.optimiser).update_layout(title="Reserve")

In [51]:
plots.plot_el_backup(res,show_margin=True).update_layout(title="Bakcup per device")

In [52]:
if 'battery' in optimiser.setDevice:
    fig=oogeso.plots.plot_device_power_energy(res,optimisation_model=simulator.optimiser,dev='battery',filename=None)
    fig.update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0)).show()